In [1]:
%pdb off
import scipy as sp
import scipy.fftpack as fftp
from scipy import pi,sin,cos

Automatic pdb calling has been turned OFF


In [2]:
sp.sin(pi)

1.2246467991473532e-16

In [3]:
a = sp.array([[sin(0),sin(2.*pi/3),sin(4.*(pi/3))],[cos(0),cos(2.*pi/3),cos(4.*(pi/3))]])

In [4]:
a

array([[ 0.       ,  0.8660254, -0.8660254],
       [ 1.       , -0.5      , -0.5      ]])

In [5]:
b = fftp.fft(a)
b

array([[  4.44089210e-16 +0.00000000e+00j,
         -2.22044605e-16 -1.50000000e+00j,
         -2.22044605e-16 +1.50000000e+00j],
       [ -2.22044605e-16 +0.00000000e+00j,
          1.50000000e+00 -5.76888806e-16j,
          1.50000000e+00 +5.76888806e-16j]])

In [6]:
omega = sp.array([0,1j,-1j])

In [7]:
omega

array([ 0.+0.j,  0.+1.j, -0.-1.j])

In [11]:
b[0,:]

array([  4.44089210e-16+0.j ,  -2.22044605e-16-1.5j,  -2.22044605e-16+1.5j])

In [10]:
b*omega

array([[  0.00000000e+00 +0.00000000e+00j,
          1.50000000e+00 -2.22044605e-16j,
          1.50000000e+00 +2.22044605e-16j],
       [ -0.00000000e+00 +0.00000000e+00j,
          5.76888806e-16 +1.50000000e+00j,
          5.76888806e-16 -1.50000000e+00j]])

In [12]:
c = fftp.ifft(b[0,:]*omega)

In [13]:
c

array([ 1.0+0.j, -0.5+0.j, -0.5+0.j])

In [30]:
a

array([[ 0.       ,  0.8660254, -0.8660254],
       [ 1.       , -0.5      , -0.5      ]])